### Fetch data

#### Load dependencies

In [29]:
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
import pandas as pd
import numpy as np
from config import SQLQuery
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_rows', 500)

In [5]:
querySno = SQLQuery('snowflake')
q = querySno

In [6]:
df = q("""WITH TRANSACTIONS AS
(
  select BUSINESS_ID,
         TRANSACTION_DATE,
         AMOUNT as all_credit_trans_amount
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='credit' and status='active'
  group by 1,2,3
),

FIRST_TRANSACTION_DATE as
(
  select  BUSINESS_ID,
          min(TRANSACTION_DATE) as FIRST_DATE
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='credit' and status='active'
  group by 1
),

APPLICATION_DETAIL as 
(
  select  APPLICATION_ID,  
          BUSINESS_ID,
          ACCOUNT_CREATE_DATE 
  from "PROD_DB"."DATA"."BUSINESSES"
  where ACCOUNT_CREATE_DATE between '2021-01-01' and '2022-02-28'
  group by 1,2,3
),

Final as (
    Select 
       FIRST_TRANSACTION_DATE.BUSINESS_ID,TRANSACTIONS.all_credit_trans_amount,
       DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE) as DAYS_SINCE_CREATED,
       APPLICATION_ID
    from TRANSACTIONS 
    inner join FIRST_TRANSACTION_DATE 
    on TRANSACTIONS.BUSINESS_ID=FIRST_TRANSACTION_DATE.BUSINESS_ID
    inner join APPLICATION_DETAIL 
    on APPLICATION_DETAIL.BUSINESS_ID=FIRST_TRANSACTION_DATE.BUSINESS_ID
    WHERE DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE)  BETWEEN 0 AND 60
)
SELECT *
FROM
(Select BUSINESS_ID, APPLICATION_ID, SUM(all_credit_trans_amount) AS all_credit_trans_amount
from Final
GROUP BY 1,2) F
LEFT JOIN "PROD_DB"."DATA"."BUSINESSES"
ON F.APPLICATION_ID = "PROD_DB"."DATA"."BUSINESSES".APPLICATION_ID""")

In [7]:
df['Deposit_Class'] = np.where(df['all_credit_trans_amount']<800, 'low', np.where(df['all_credit_trans_amount']>1200, 'high', 'mid'))
df['Deposit_Class'].value_counts()

low     53190
high    49719
mid      7733
Name: Deposit_Class, dtype: int64